# Classificação de imagens com Convolutional Neural Networks

## Imports

In [ ]:
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

## Análise Exploratória (EDA)

### Carregar dataset

In [ ]:
# Carregar o dataset
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

### Lista de classes

In [ ]:
# Lista de classes
classes = ['airplane', 'automobile', 'bird', 'cat', 'deer',
           'dog', 'frog', 'horse', 'ship', 'truck']

classes

### Mostrar 10 imagens aleatórias com labels

In [ ]:
plt.figure(figsize=(6, 4))
for i in range(10):
    idx = np.random.randint(0, len(x_train))
    plt.subplot(2, 5, i+1)
    plt.imshow(x_train[idx])
    plt.title(classes[int(y_train[idx])])
    plt.axis('off')
plt.tight_layout()
plt.show()

## Pré-Processamento

### Normalizar os dados (de 0-255 para 0-1)

In [ ]:
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0 

### Converter os rótulos para one-hot encoding

In [ ]:
y_train_cat = to_categorical(y_train, 10)
y_test_cat = to_categorical(y_test, 10)

### Verificar dimensões

In [ ]:
x_train.shape

In [ ]:
y_train_cat.shape

## Criando o modelo CNN

### Função importante

In [ ]:
def plots(history):
    # Plotar os gráficos 2x2
    history_dict = history.history
    fig, axs = plt.subplots(2, 2, figsize=(12, 8))

    # Acurácia - Treinamento
    axs[0, 0].plot(history_dict['accuracy'], label='Treinamento')
    axs[0, 0].set_title('Acurácia - Treino')
    axs[0, 0].set_xlabel('Época')
    axs[0, 0].set_ylabel('Acurácia')
    axs[0, 0].grid(True)
    axs[0, 0].legend()

    # Acurácia - Validação
    axs[0, 1].plot(history_dict['val_accuracy'], label='Validação', color='orange')
    axs[0, 1].set_title('Acurácia - Validação')
    axs[0, 1].set_xlabel('Época')
    axs[0, 1].set_ylabel('Acurácia')
    axs[0, 1].grid(True)
    axs[0, 1].legend()

    # Loss - Treinamento
    axs[1, 0].plot(history_dict['loss'], label='Treinamento', color='green')
    axs[1, 0].set_title('Loss - Treino')
    axs[1, 0].set_xlabel('Época')
    axs[1, 0].set_ylabel('Loss')
    axs[1, 0].grid(True)
    axs[1, 0].legend()

    # Loss - Validação
    axs[1, 1].plot(history_dict['val_loss'], label='Validação', color='red')
    axs[1, 1].set_title('Loss - Validação')
    axs[1, 1].set_xlabel('Época')
    axs[1, 1].set_ylabel('Loss')
    axs[1, 1].grid(True)
    axs[1, 1].legend()

    plt.tight_layout()
    plt.show()

### Definindo arquitetura da CNN comum

In [ ]:
def CNN_CIFAR10(x_train, y_train, x_test, y_test, epochs, sumarity=True):
    # Definir a arquitetura do modelo
    model = Sequential()

    # 1ª camada convolucional
    model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    # 2ª camada convolucional
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    # 3ª camada convolucional
    model.add(Conv2D(128, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    # Achatar e adicionar densa
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))  # Regularização

    # Saída - 10 classes (CIFAR-10)
    model.add(Dense(10, activation='softmax'))

    # Compilar o modelo
    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

    # Callbacks
    early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

    # Resumo da arquitetura
    if sumarity:
        model.summary()
        return model

    # Treinamento do modelo
    history = model.fit(
        x_train, y_train,
        epochs=epochs,
        batch_size=64,
        validation_data=(x_test, y_test),
        callbacks=[early_stop]
    )

    #Plotando gráficos
    plots(history)

    return model

CNN_CIFAR10(0, 0, 0, 0, 0)

### Definindo arquitetura da CNN com data argumentation

In [ ]:
def CNN_CIFAR10(x_train, y_train, x_test, y_test, epochs, summary=True, use_augmentation=False):

    model = Sequential()

    # 1º bloco conv
    model.add(Conv2D(32, (3, 3), padding='same', activation='relu', input_shape=(32, 32, 3)))
    model.add(BatchNormalization())
    model.add(Conv2D(32, (3, 3), padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    # 2º bloco conv
    model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    # 3º bloco conv
    model.add(Conv2D(128, (3, 3), padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(Conv2D(128, (3, 3), padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.4))

    # Flatten + Dense
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))
    model.add(Dense(10, activation='softmax'))

    # Compilação
    model.compile(
        optimizer=Adam(learning_rate=0.001),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )

    if summary:
        model.summary()
        return model

    # Callbacks
    early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

    if use_augmentation:
        print("Usando Data Augmentation...")
        datagen = ImageDataGenerator(
            rotation_range=15,
            width_shift_range=0.1,
            height_shift_range=0.1,
            horizontal_flip=True
        )
        datagen.fit(x_train)

        history = model.fit(
            datagen.flow(x_train, y_train, batch_size=64),
            epochs=epochs,
            validation_data=(x_test, y_test),
            callbacks=[early_stop]
        )
    else:
        history = model.fit(
            x_train, y_train,
            batch_size=64,
            epochs=epochs,
            validation_data=(x_test, y_test),
            callbacks=[early_stop]
        )

    #Plotando gráficos
    plots(history)

    return model

CNN_CIFAR10(0, 0, 0, 0, 0, summary=True, use_augmentation=False)

### Treinando o modelo CNN comum

In [ ]:
model = CNN_CIFAR10(x_train, y_train_cat, x_test, y_test_cat, 20, sumarity=False)
test_loss, test_acc = model.evaluate(x_test, y_test_cat, verbose=2)
print('Erro:', test_loss, 'Acurácia:', test_acc)

### Treinando CNN com data argumentation

In [ ]:
model =  CNN_CIFAR10(x_train, y_train_cat, x_test, y_test_cat, 20, summary=False, use_augmentation=True)
test_loss, test_acc = model.evaluate(x_test, y_test_cat, verbose=2)
print('Erro:', test_loss, 'Acurácia:', test_acc)